In [1]:
from bokeh.plotting import figure, output_file, show

from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

## 수집된 데이터에 대한 간략한 설명

 2016년 5월부터 2017년 12월 넷째주까지 주요 13개 언론사의 기사중 연애와 스포츠를 제외한 기사들을 수집. 해당 기사들의 네이버 URL과 언론사의 URL을 기준으로 페이스북의 Graph api로 조회하여 페이스북에서의 공유수, 코멘트수(페이스북자체 댓글, 페이스북 댓글 플러그인), 반응(좋아요, 최고예요, 웃겨요, 멋저요, 슬퍼요, 화나요의 합)수를 정리함. 
 
최초 18개의 언론사의 기사를 수집했으나, 데이터 정합성에 문제의 소지가 있거나 2017년 중 데이터를 수집한 언론사는 우선 제외함. 언론사 목록은 아래에 표시. 

> * 조선일보
> * 동아일보
> * 이데일리
> * 한겨레신문
> * 중앙일보
> * JTBC
> * KBS
> * 경향신문
> * MBC
> * 매일경제
> * ~~머니투데이~~(2017년 중반부터 수집)
> * ~~뉴시스~~(2017년 중반부터 수집)
> * ~~노컷뉴스~~(언론사 URL 변경으로 인한 데이터 오류 가능성)
> * 오마이뉴스
> * SBS
> * ~~서울경제~~(2017년 중반부터 수집)
> * 연합뉴스
> * ~~YTN~~(언론사 URL 변경으로 인한 데이터 오류 가능성)

## 수집된 데이터 정리

In [2]:
import glob

files = glob.glob('./newsdata/data/*.json')

In [3]:
import json
def read_json_file(path_to_file):
    with open(path_to_file) as p:
        rows=[json.loads(line) for line in p]
    return rows

In [4]:
import pandas as pd

data={}
for path in files:
    file = path.split('/')[3]
    media = file.split('.')[0]
    if media not in data:
        data[media] = read_json_file(path)
    else :
        data[media].extend(read_json_file(path))    
    

#df = pd.DataFrame(data)

In [5]:
dict_of_df = {k: pd.DataFrame(v) for k,v in data.items()}
df = pd.concat(dict_of_df, axis=0)
df.reset_index(level=0, inplace=True)
len(df)

1045920

In [6]:
df=df.rename({'level_0':'media'}, axis=1)

In [7]:
df['year'] = df['date'].apply(lambda x: x[0] )
df['month'] = df['date'].apply(lambda x: x[1] )
df['datetime'] = df['date'].apply(lambda x: "-".join(x) )
df['datetime'] = pd.to_datetime(df['datetime'])

In [8]:
df['n_comment_count']=df['n_engagement'].apply(lambda x:x['comment_count'] if(isinstance(x, dict)) else 0)
df['n_comment_plugin_count']=df['n_engagement'].apply(lambda x:x['comment_plugin_count'] if(isinstance(x, dict)) else 0)
df['n_share_count']=df['n_engagement'].apply(lambda x:x['share_count'] if(isinstance(x, dict)) else 0)
df['n_reaction_count']=df['n_engagement'].apply(lambda x:x['reaction_count'] if(isinstance(x, dict)) else 0)
df['o_comment_count']=df['o_engagement'].apply(lambda x:x['comment_count'] if(isinstance(x, dict)) else 0)
df['o_comment_plugin_count']=df['o_engagement'].apply(lambda x:x['comment_plugin_count'] if(isinstance(x, dict)) else 0)
df['o_share_count']=df['o_engagement'].apply(lambda x:x['share_count'] if(isinstance(x, dict)) else 0)
df['o_reaction_count']=df['o_engagement'].apply(lambda x:x['reaction_count'] if(isinstance(x, dict)) else 0)

In [9]:
df['tot_comment_count']=df['n_comment_count']+df['o_comment_count']
df['tot_plugin_count']=df['n_comment_plugin_count']+df['o_comment_plugin_count']
df['tot_share_count']=df['n_share_count']+df['o_share_count']
df['tot_reaction_count']=df['n_reaction_count']+df['o_reaction_count']
df['tot_all_comment']=df['tot_comment_count']+df['tot_plugin_count']

## 조사기간 한정 및 오류 가능성이 있는 언론사 제외

데이터 설명에서 언급된 바와 같이 최초 수집된 18개 언론사중 머니투데이, 뉴시스, 서울경제는 2017년중에 데이터를 수집한 관계로 제외하고 노컷뉴스와 YTN은 해당 회사의 url 정책 변경등으로 데이터 정합성을 보장하기 어려워 제외함.

In [10]:
df=df[~df.media.isin(['mt','newsis', 'sedaily', 'nocut', 'ytn'])]
df_year = df[(df['datetime']>'20160430') & (df['datetime']<'20171225')]

## 월별 언론사별 뉴스 공유수

공유수란 이용자가 해당 언론사 홈페이지나 페이스북에서 공유 기능을 활용하여 사용자 자신의 타임라인에 해당 기사를 공유할때 발생하는 숫자의 합산.

In [11]:
media_share_sum = df_year.groupby('media')['tot_share_count'].sum()
sort_sum=media_share_sum.sort_values(ascending=False)
sort_sum

media
yonhap      1417660
hani        1414191
ohmynews    1257679
chosun      1157615
kh           988565
joins        725679
jtbc         540296
sbs          535557
mk           272720
kbs          219985
donga        204766
edaily       147788
mbc          133274
Name: tot_share_count, dtype: int64

In [12]:
share_group_sum=df_year.groupby([pd.Grouper(key='datetime', freq='M'), 'media'])['tot_share_count'].sum()
unstacked_share_sum=share_group_sum.unstack(level=-1)
unstacked_share_sum.reset_index(level=0, inplace=True)
#unstacked_share_mean

In [13]:
p1 = figure(plot_width=750, plot_height=500,x_axis_type='datetime', title="월별 공유수 합산")
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = '월별'
p1.yaxis.axis_label = '공유수 합산'
p1.line(unstacked_share_sum['datetime'], unstacked_share_sum['chosun'], color='#f90202', legend='조선일보',line_width=2)
p1.line(unstacked_share_sum['datetime'], unstacked_share_sum['joins'], color='#E2A00B', legend='중앙일보',line_width=2)
p1.line(unstacked_share_sum['datetime'], unstacked_share_sum['yonhap'], color='#1cefef', legend='연합뉴스',line_width=2, line_dash='dashed')
p1.line(unstacked_share_sum['datetime'], unstacked_share_sum['ohmynews'], color='#ff3300', legend='오마이뉴스',line_width=2, line_dash='dashed')
p1.line(unstacked_share_sum['datetime'], unstacked_share_sum['hani'], color='#01996e', legend='한겨레',line_width=2)
p1.line(unstacked_share_sum['datetime'], unstacked_share_sum['kh'], color='#18407D', legend='경향신문',line_width=2)
p1.line(unstacked_share_sum['datetime'], unstacked_share_sum['jtbc'], color='#F48179', legend='JTBC',line_width=2, line_dash='dotted')
p1.line(unstacked_share_sum['datetime'], unstacked_share_sum['sbs'], color='#1D3A6E', legend='SBS',line_width=2, line_dash='dotted')



show(p1)

In [14]:
media_share_mean = df_year.groupby('media')['tot_share_count'].mean()
sort_mean=media_share_mean.sort_values(ascending=False)
sort_mean

media
ohmynews    34.007869
hani        29.170005
jtbc        21.916924
kh          16.699579
chosun      15.402214
joins       10.546587
sbs          8.559462
mbc          6.518022
yonhap       6.492158
donga        5.744593
mk           5.034893
kbs          4.510940
edaily       4.047212
Name: tot_share_count, dtype: float64

In [15]:
share_group_mean=df_year.groupby([pd.Grouper(key='datetime', freq='M'), 'media'])['tot_share_count'].mean()
unstacked_share_mean=share_group_mean.unstack(level=-1)
unstacked_share_mean.reset_index(level=0, inplace=True)

In [16]:
p1 = figure(plot_width=750, plot_height=500,x_axis_type='datetime', title="월별 기사당 공유수 평균")
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = '월별'
p1.yaxis.axis_label = '기사당 공유수 평균'
p1.line(unstacked_share_mean['datetime'], unstacked_share_mean['chosun'], color='#f90202', legend='조선일보',line_width=2)
p1.line(unstacked_share_mean['datetime'], unstacked_share_mean['joins'], color='#E2A00B', legend='중앙일보',line_width=2)
p1.line(unstacked_share_mean['datetime'], unstacked_share_mean['yonhap'], color='#1cefef', legend='연합뉴스',line_width=2, line_dash='dashed')
p1.line(unstacked_share_mean['datetime'], unstacked_share_mean['ohmynews'], color='#ff3300', legend='오마이뉴스',line_width=2, line_dash='dashed')
p1.line(unstacked_share_mean['datetime'], unstacked_share_mean['hani'], color='#01996e', legend='한겨레',line_width=2)
p1.line(unstacked_share_mean['datetime'], unstacked_share_mean['kh'], color='#18407D', legend='경향신문',line_width=2)
p1.line(unstacked_share_mean['datetime'], unstacked_share_mean['jtbc'], color='#F48179', legend='JTBC',line_width=2, line_dash='dotted')
p1.line(unstacked_share_mean['datetime'], unstacked_share_mean['sbs'], color='#1D3A6E', legend='SBS',line_width=2, line_dash='dotted')

show(p1)

## 월별 언론사별 반응수 합산

반응수란 페이스북내에서 링크된 콘텐츠의 하단에 있는 버튼(좋아요, 최고예요, 웃겨요, 멋저요, 슬퍼요, 화나요)을 활용해 이용자가 감정적 반응을 한 수의 합산값. 현재 개별값은 제공하지 않고 있음.

In [17]:
media_reaction_sum = df_year.groupby('media')['tot_reaction_count'].sum()
sort_reaction_total=media_reaction_sum.sort_values(ascending=False)
sort_reaction_total

media
hani        18313795
ohmynews    17315667
kh          16516102
joins       16192627
sbs         12877872
yonhap      11272402
chosun      10552906
jtbc        10039417
mk           1598513
kbs          1435025
donga        1388034
edaily       1156878
mbc           650415
Name: tot_reaction_count, dtype: int64

In [18]:
reaction_group_sum=df_year.groupby([pd.Grouper(key='datetime', freq='M'), 'media'])['tot_reaction_count'].sum()
unstacked_reaction_sum=reaction_group_sum.unstack(level=-1)
unstacked_reaction_sum.reset_index(level=0, inplace=True)

In [19]:
p1 = figure(plot_width=750, plot_height=500,x_axis_type='datetime', title="월별 반응수 합산")
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = '월별'
p1.yaxis.axis_label = '반응수 합산'
p1.line(unstacked_reaction_sum['datetime'], unstacked_reaction_sum['chosun'], color='#f90202', legend='조선일보',line_width=2)
p1.line(unstacked_reaction_sum['datetime'], unstacked_reaction_sum['joins'], color='#E2A00B', legend='중앙일보',line_width=2)
p1.line(unstacked_reaction_sum['datetime'], unstacked_reaction_sum['yonhap'], color='#1cefef', legend='연합뉴스',line_width=2, line_dash='dashed')
p1.line(unstacked_reaction_sum['datetime'], unstacked_reaction_sum['ohmynews'], color='#ff3300', legend='오마이뉴스',line_width=2, line_dash='dashed')
p1.line(unstacked_reaction_sum['datetime'], unstacked_reaction_sum['hani'], color='#01996e', legend='한겨레',line_width=2)
p1.line(unstacked_reaction_sum['datetime'], unstacked_reaction_sum['kh'], color='#18407D', legend='경향신문',line_width=2)
p1.line(unstacked_reaction_sum['datetime'], unstacked_reaction_sum['jtbc'], color='#F48179', legend='JTBC',line_width=2, line_dash='dotted')
p1.line(unstacked_reaction_sum['datetime'], unstacked_reaction_sum['sbs'], color='#1D3A6E', legend='SBS',line_width=2, line_dash='dotted')

show(p1)

In [20]:
media_reaction_mean = df_year.groupby('media')['tot_reaction_count'].mean()
sort_reaction_mean=media_reaction_mean.sort_values(ascending=False)
sort_reaction_mean

media
ohmynews    468.218782
jtbc        407.245538
hani        377.752006
kh          279.002348
joins       235.334007
sbs         205.818728
chosun      140.407749
yonhap       51.621835
donga        38.940497
mbc          31.809801
edaily       31.681400
mk           29.511372
kbs          29.426149
Name: tot_reaction_count, dtype: float64

In [21]:
reaction_group_mean=df_year.groupby([pd.Grouper(key='datetime', freq='M'), 'media'])['tot_reaction_count'].mean()
unstacked_reaction_mean=reaction_group_mean.unstack(level=-1)
unstacked_reaction_mean.reset_index(level=0, inplace=True)

In [22]:
p1 = figure(plot_width=750, plot_height=500,x_axis_type='datetime', title="월별 기사당 반응수 평균")
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = '월별'
p1.yaxis.axis_label = '기사당 반응수 평균'
p1.line(unstacked_reaction_mean['datetime'], unstacked_reaction_mean['chosun'], color='#f90202', legend='조선일보',line_width=2)
p1.line(unstacked_reaction_mean['datetime'], unstacked_reaction_mean['joins'], color='#E2A00B', legend='중앙일보',line_width=2)
p1.line(unstacked_reaction_mean['datetime'], unstacked_reaction_mean['yonhap'], color='#1cefef', legend='연합뉴스',line_width=2, line_dash='dashed')
p1.line(unstacked_reaction_mean['datetime'], unstacked_reaction_mean['ohmynews'], color='#ff3300', legend='오마이뉴스',line_width=2, line_dash='dashed')
p1.line(unstacked_reaction_mean['datetime'], unstacked_reaction_mean['hani'], color='#01996e', legend='한겨레',line_width=2)
p1.line(unstacked_reaction_mean['datetime'], unstacked_reaction_mean['kh'], color='#18407D', legend='경향신문',line_width=2)
p1.line(unstacked_reaction_mean['datetime'], unstacked_reaction_mean['jtbc'], color='#F48179', legend='JTBC',line_width=2, line_dash='dotted')
p1.line(unstacked_reaction_mean['datetime'], unstacked_reaction_mean['sbs'], color='#1D3A6E', legend='SBS',line_width=2, line_dash='dotted')

show(p1)

## 월별 언론사별 코멘트 수 합산


In [23]:
all_comment_sum = df_year.groupby('media')['tot_all_comment'].sum()
sort_comment_total=all_comment_sum.sort_values(ascending=False)
sort_comment_total

media
sbs         2497177
chosun      2136996
joins       2089655
yonhap      1876219
hani        1510557
kh          1393676
ohmynews    1290359
jtbc         737172
kbs          237487
mk           233889
donga        202458
edaily       194901
mbc          116782
Name: tot_all_comment, dtype: int64

In [24]:
comment_group_sum=df_year.groupby([pd.Grouper(key='datetime', freq='M'), 'media'])['tot_all_comment'].sum()
unstacked_comment_sum=comment_group_sum.unstack(level=-1)
unstacked_comment_sum.reset_index(level=0, inplace=True)

In [25]:
p1 = figure(plot_width=750, plot_height=500,x_axis_type='datetime', title="월별 댓글수 합산")
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = '월별'
p1.yaxis.axis_label = '댓글수 평균'
p1.line(unstacked_comment_sum['datetime'], unstacked_comment_sum['chosun'], color='#f90202', legend='조선일보',line_width=2)
p1.line(unstacked_comment_sum['datetime'], unstacked_comment_sum['joins'], color='#E2A00B', legend='중앙일보',line_width=2)
p1.line(unstacked_comment_sum['datetime'], unstacked_comment_sum['yonhap'], color='#1cefef', legend='연합뉴스',line_width=2, line_dash='dashed')
p1.line(unstacked_comment_sum['datetime'], unstacked_comment_sum['ohmynews'], color='#ff3300', legend='오마이뉴스',line_width=2, line_dash='dashed')
p1.line(unstacked_comment_sum['datetime'], unstacked_comment_sum['hani'], color='#01996e', legend='한겨레',line_width=2)
p1.line(unstacked_comment_sum['datetime'], unstacked_comment_sum['kh'], color='#18407D', legend='경향신문',line_width=2)
p1.line(unstacked_comment_sum['datetime'], unstacked_comment_sum['jtbc'], color='#F48179', legend='JTBC',line_width=2, line_dash='dotted')
p1.line(unstacked_comment_sum['datetime'], unstacked_comment_sum['sbs'], color='#1D3A6E', legend='SBS',line_width=2, line_dash='dotted')

show(p1)

In [26]:
all_comment_mean = df_year.groupby('media')['tot_all_comment'].mean()
sort_comment_mean=all_comment_mean.sort_values(ascending=False)
sort_comment_mean

media
sbs         39.910771
ohmynews    34.891542
hani        31.157711
joins       30.369802
jtbc        29.903132
chosun      28.433002
kh          23.543017
yonhap       8.592123
mbc          5.711449
donga        5.679843
edaily       5.337414
kbs          4.869830
mk           4.318004
Name: tot_all_comment, dtype: float64

In [27]:
comment_group_mean=df_year.groupby([pd.Grouper(key='datetime', freq='M'), 'media'])['tot_all_comment'].mean()
unstacked_comment_mean=comment_group_mean.unstack(level=-1)
unstacked_comment_mean.reset_index(level=0, inplace=True)

In [28]:
p1 = figure(plot_width=750, plot_height=500,x_axis_type='datetime', title="월별 기사당 댓글수 평균")
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = '월별'
p1.yaxis.axis_label = '기사당 댓글수 평균'
p1.line(unstacked_comment_mean['datetime'], unstacked_comment_mean['chosun'], color='#f90202', legend='조선일보',line_width=2)
p1.line(unstacked_comment_mean['datetime'], unstacked_comment_mean['joins'], color='#E2A00B', legend='중앙일보',line_width=2)
p1.line(unstacked_comment_mean['datetime'], unstacked_comment_mean['yonhap'], color='#1cefef', legend='연합뉴스',line_width=2, line_dash='dashed')
p1.line(unstacked_comment_mean['datetime'], unstacked_comment_mean['ohmynews'], color='#ff3300', legend='오마이뉴스',line_width=2, line_dash='dashed')
p1.line(unstacked_comment_mean['datetime'], unstacked_comment_mean['hani'], color='#01996e', legend='한겨레',line_width=2)
p1.line(unstacked_comment_mean['datetime'], unstacked_comment_mean['kh'], color='#18407D', legend='경향신문',line_width=2)
p1.line(unstacked_comment_mean['datetime'], unstacked_comment_mean['jtbc'], color='#F48179', legend='JTBC',line_width=2, line_dash='dotted')
p1.line(unstacked_comment_mean['datetime'], unstacked_comment_mean['sbs'], color='#1D3A6E', legend='SBS',line_width=2, line_dash='dotted')

show(p1)